In [18]:
library(readr)
library(dplyr)
library(tidyr)
library(stringr)
library(lfe)
library(ggsci)
library(lubridate)
library(ggplot2)
library(doMC)
library(gridExtra)
library(urbnmapr)
library(viridis)
library(ggthemes)
library(ggsci)
library(ggpubr)
library(stargazer)
library(scales)
registerDoMC(4)
source('./adjacency_and_cluster_robust_inference.R')


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘lmtest’


The following object is masked from ‘package:lfe’:

    waldtest




In [2]:
fp  <- '/pool001/mfzhao/'
df  <- read_rds(str_c(fp, '/PROCESSED_DATA/panel_pre_xgr.RDS'))
WM <- read_rds(str_c(fp, 'PROCESSED_DATA/stateWM.RDS'))

In [13]:
state_code_from_fips <- function(x) {
    floor(as.integer(as.character(x))/1000)
}

conley_selector_matrix <- WM
conley_selector_matrix[outer(state_code_from_fips(colnames(conley_selector_matrix)),
                             state_code_from_fips(colnames(conley_selector_matrix)), "==")] <- 1

,01001,01003,01005,01007,01009,01011,01013,01015,01017,01019,⋯,56013,56015,56021,56023,56025,56029,56033,56037,56039,56041
01001,1,1,1,1,1,1,1,1,1,1,⋯,6.664016e-05,1.802524e-05,9.771954e-04,7.823131e-05,1.621731e-04,5.803504e-05,7.584890e-05,6.967705e-05,9.876657e-05,3.187276e-05
01003,1,1,1,1,1,1,1,1,1,1,⋯,7.968581e-05,2.650056e-05,2.036968e-04,3.133476e-05,1.458002e-04,1.041010e-04,4.925007e-05,5.976556e-05,1.125024e-04,3.720084e-05
01005,1,1,1,1,1,1,1,1,1,1,⋯,3.491179e-05,7.891413e-06,1.184503e-04,1.331018e-05,4.867162e-05,4.386717e-05,1.478505e-05,2.043278e-05,2.055038e-05,2.892720e-05
01007,1,1,1,1,1,1,1,1,1,1,⋯,6.684583e-05,2.948060e-05,2.657146e-04,3.300755e-05,1.528929e-04,1.115139e-04,1.534448e-04,9.681391e-05,3.839705e-05,3.462836e-05
01009,1,1,1,1,1,1,1,1,1,1,⋯,9.104569e-05,3.322531e-05,4.349890e-04,6.778772e-05,3.314172e-04,1.593819e-04,1.522969e-04,1.319642e-04,5.091187e-05,5.330131e-05
01011,1,1,1,1,1,1,1,1,1,1,⋯,8.039159e-06,4.130657e-06,1.074741e-04,5.733078e-06,3.843506e-05,1.781666e-05,1.533276e-05,1.659126e-05,1.883327e-05,5.631508e-06
01013,1,1,1,1,1,1,1,1,1,1,⋯,1.431805e-04,1.276985e-05,1.263264e-04,1.948431e-05,1.277780e-04,1.419324e-04,2.058684e-05,4.353074e-05,4.150330e-05,1.837197e-05
01015,1,1,1,1,1,1,1,1,1,1,⋯,6.187725e-05,4.359535e-05,2.979338e-04,6.135234e-05,1.217496e-04,1.072783e-04,4.468032e-05,5.616771e-05,3.772726e-05,2.313554e-05
01017,1,1,1,1,1,1,1,1,1,1,⋯,2.275962e-05,7.357994e-06,7.202998e-05,5.169813e-06,5.704994e-05,2.206495e-05,1.464893e-05,1.877373e-05,1.576660e-05,8.138450e-06
01019,1,1,1,1,1,1,1,1,1,1,⋯,5.969369e-05,1.413192e-05,2.220011e-04,1.741330e-04,9.160539e-05,7.419483e-05,4.188157e-05,1.769567e-04,3.644311e-05,1.727828e-05


In [30]:
df %>%
    select(key,
           date, 
           cluster,
           n,
           log_mcbgv,
           log_pnchd,
           log_pgt1hafh, 
           log_ppthlt75,
           log_pgt2kmt,
           matches('new'),
           matches('unemp'),
           matches('^log_stalter_'), 
           matches('^[pt][rm][ca][px][01][0-9]$'),
           matches('^p[123]...$'),
           matches('stalter_p[123]...$'),
           matches('stalter[NF]'),
           matches('stalter_[pt][rm][ca][px][01][0-9]$'),
           matches('as_(st)?new')) -> df

ERROR: Error: Column `state` must be length 470106 (the number of rows) or one, not 2


In [20]:
cns <- colnames(df)
cns

[1] "key"                  "date"                 "cluster"             
 [4] "n"                    "log_mcbgv"            "log_pnchd"           
 [7] "log_pgt1hafh"         "log_ppthlt75"         "log_pgt2kmt"         
[10] "newcases"             "newdeaths"            "stnewcases"          
[13] "stnewdeaths"          "as_newcases"          "as_newdeaths"        
[16] "as_stnewcases"        "as_stnewdeaths"       "ct_unemp_rate"       
[19] "st_unemp_rate"        "log_stalter_mcbgv"    "log_stalter_pnchd"   
[22] "log_stalter_pgt1hafh" "log_stalter_ppthlt75" "log_stalter_pgt2kmt" 
[25] "prcp01"               "prcp02"               "prcp03"              
[28] "prcp04"               "prcp05"               "prcp06"              
[31] "prcp07"               "prcp08"               "prcp09"              
[34] "prcp10"               "tmax02"               "tmax03"              
[37] "tmax04"               "tmax05"               "tmax06"              
[40] "tmax07"               "tmax08"               "tmax09"              
[43] "tmax10"               "p1sdp"                "p2shp"               
[46] "p3rop"                "stalter_p1sdp"        "stalter_p2shp"       
[49] "stalter_p3rop"        "stalterNear_p1sdp"    "stalterNear_p2shp"   
[52] "stalterNear_p3rop"    "stalterFar_p1sdp"     "stalterFar_p2shp"    
[55] "stalterFar_p3rop"     "stalter_prcp01"       "stalter_prcp02"      
[58] "stalter_prcp03"       "stalter_prcp04"       "stalter_prcp05"      
[61] "stalter_prcp06"       "stalter_prcp07"       "stalter_prcp08"      
[64] "stalter_prcp09"       "stalter_prcp10"       "stalter_tmax02"      
[67] "stalter_tmax03"       "stalter_tmax04"       "stalter_tmax05"      
[70] "stalter_tmax06"       "stalter_tmax07"       "stalter_tmax08"      
[73] "stalter_tmax09"       "stalter_tmax10"

In [21]:
ldvs <- c('log_mcbgv', 'log_pgt2kmt', 'log_pgt1hafh', 'log_pnchd')
dvs  <- c('mcbgv', 'pnchd', 'pgt1hafh', 'pgt2kmt')

In [22]:
weightedAlters <- function(df, wm, ...) {
    df %>% 
        select(date, key, ...) %>%
        spread(key = key, value = ...) %>%
        ungroup() %>%
        select(-date) %>%
        as.matrix() -> txn_data
    
    df %>%
        ungroup() %>%
        select(date) %>%
        distinct() %>%
        arrange(date) -> dates
    
    outMatrix <- tcrossprod(txn_data, wm)
    colnames(outMatrix) <- colnames(txn_data)
    
    data.frame(dates, outMatrix) %>%
        gather(key = 'key', value = 'value', -date) %>%
        arrange(date, key) %>%
        select(-date, -key) -> out_df
    return(out_df$value)
}

In [94]:
get_coefs <- function(dv, vars, ivmod = F, cluster_robust_se = F, ...) {
    lhs <- str_c(dv, ' ~ ')
    rhs <- str_c(vars, '| key + date | ')

    if(ivmod == T) {
        alter_weather <- str_c(cns[str_detect(cns, '^stalter_[pt][rm][ca][px][01][0-9]$')], collapse = ' + ')
        iv <- ifelse(str_detect(dv, 'log'), str_replace(dv, 'log', 'log_stalter'), str_c('alter_', dv))
        ivf <- str_c('(', iv, ' ~ ', alter_weather, ') | cluster')
        rhs <- str_c(rhs, ivf)
    } else {
        rhs <- str_c(rhs, '0 | cluster')
    }
    
    form  <- as.formula(str_c(lhs, rhs))
    
    if(cluster_robust_se == F) {
        model <- felm(form, df, weights = df$n, ...)
        
        out <- data.frame(coef = coef(model),
                          se = sqrt(diag(model$clustervcv)),
                          var = rownames(model$coef),
                          model = rep('State-clustered S.E.', length(model$coef)),
                          dv = rep(dv, length(model$coef))) %>% 
        filter(!str_detect(var, '[pPtT][rRmM][cCaA][pPxX]'))
    
    } else {
        model <- felm(form, df, weights = df$n, keepCX = TRUE, ...)
        ses <- summarize.felm.fit.with.adj(fm= model, conley_selector_matrix, df$cluster, n_ds=182)
        
        out <- data.frame(coef = coef(model),
                          se = sqrt(diag(ses$vcov.both)),
                          var = rownames(model$coef),
                          model = rep('Cluster- and adjacency-robust S.E.', length(model$coef)),
                          dv = rep(dv, length(model$coef))) %>% 
        filter(!str_detect(var, '[pPtT][rRmM][cCaA][pPxX]'))
    }
    
    #colnames(out) <- c('model', 'dv', 'var', 'estimate', 'se', 't', 'p-val')
    return(out)
}

In [25]:
ctrls <- '+ prcp01 + prcp02 + prcp03 + prcp04 + prcp05 + prcp06 + prcp07 + prcp08 + prcp09 + prcp10 + 
            tmax02 + tmax03 + tmax04 + tmax05 + tmax06 + tmax07 + tmax08 + tmax09 + tmax10 + 
            as_newcases + as_newdeaths + as_stnewcases + as_stnewdeaths + ct_unemp_rate + st_unemp_rate '
ctrls2 <- '+ as_stalter_stnewcases + as_stalter_stnewdeaths + stalter_st_unemp_rate'
 
f1    <- str_c(' p1sdp + p2shp + p3rop + stalter_p1sdp + stalter_p2shp + stalter_p3rop ', ctrls)

In [ ]:
foreach(dv = ldvs, .combine = rbind) %:% 
    foreach(cluster_robust_se = c(TRUE, FALSE), .combine = rbind) %dopar%
    get_coefs(dv, f1, F, cluster_robust_se) -> coefs

In [106]:
pn <- c('p1sdp' = "Initial Policies",
        'p2shp' = "Shelter-in-place",
        'p3rop' = "Reopening ",
        'A' = "Ego State Policy",
        'B' = "Alter State Policy")

compare_ses <- coefs %>%
    filter(str_detect(var, 'p[123]')) %>%
    mutate(model = ifelse(model == 'State-clustered S.E.', 
                          'State-clustered\nS.E.', 'Cluster- and\nadjacency-robust S.E.')) %>%
    mutate(var = str_replace(var, 'p[23]$', 'p'),
           dv  = factor(dv, levels = c('log_mcbgv', 'log_pgt2kmt', 'log_pgt1hafh', 'log_pnchd')),
           model = factor(model, levels = c('State-clustered\nS.E.', 'Cluster- and\nadjacency-robust S.E.')),
           type = ifelse(str_detect(var, 'stalter'), 'B', 'A'),
           var = str_replace(var, 'stalter_', '')) %>%
    ggplot(aes(x = model, y = coef, color = dv)) + 
    geom_point(position = position_dodge(width = 0.5)) + 
    geom_linerange(aes(ymin = coef - 1.98 * se,
                       ymax = coef + 1.98 * se),
                   position = position_dodge(width = 0.5)) +
    facet_grid(type~var, scales = 'free', labeller = as_labeller(pn)) +
    scale_color_d3(labels = c('Average number of\nlocations visited', 
                              'Traveling more\nthan 2km', 
                              'More than 1 hour\naway from home', 
                              'Leaving home')) +
    xlab('') + 
    ylab('Coefficient Estimates') +
    labs(color = 'Outcome') +
    geom_hline(aes(yintercept = 0), linetype = 2) +
    theme_light() + 
    theme(legend.position = 'bottom')

In [108]:
ggsave(compare_ses, file='../output/compare_ses.pdf', width=7, height=5, units='in')